In [134]:
import numpy as np
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv

# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'

In [135]:
def plot_cameras_and_axes(ax, C_arr, axis_arr, s):

    col = cm.rainbow(np.linspace(0, 1, np.size(C_arr,1)))

    for i in range(np.size(C_arr,1)):

        C = C_arr[:,i]
        axis = axis_arr[:,i]

        ax.plot(C[0], C[1], C[2], 'o', color=col[i],  label='Camera {}'.format(i+1), alpha=0.7)

        x_axis = C[0] + s*axis[0]
        y_axis = C[1] + s*axis[1]
        z_axis = C[2] + s*axis[2]

        # ax.plot(x_axis, y_axis, z_axis, 'o', label='Axis')
        ax.plot([x_axis, C[0]], [y_axis, C[1]], [z_axis, C[2]], '-', color=col[i], lw=3, alpha=0.7)

In [136]:
def plot_cameras_and_3D_points(X, C_arr, axis_arr, s, path):
    
    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection='3d')

    ax.plot(X[0], X[1], X[2], '.', ms=0.6, color='magenta', label='3D points')
    plot_cameras_and_axes(ax, C_arr, axis_arr, s)

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.axis('equal')

    plt.legend(loc="lower right")
    fig.savefig(path, dpi=300)

    plt.show()
    

In [137]:
def plot_image_points_projected_points_and_image(x_proj, x_img, img, path):

    fig = plt.figure(figsize=(10,8))

    plt.plot(x_img[0], x_img[1], 'D', color='blue', ms=1.2, label='Image points')
    plt.plot(x_proj[0], x_proj[1], 's', color='red', ms=0.4, label='Projected points')

    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.axis('equal')
    plt.gca().invert_yaxis()

    plt.legend(loc="lower right")
    plt.imshow(img)
    fig.savefig(path, dpi=300)

    plt.show()

In [138]:
def rotate_and_translate_P2_points(x, theta, x_trans, y_trans):

    cos = np.cos(theta)
    sin= np.sin(theta)
    R = np.array([[cos, -sin, x_trans],[sin, cos, y_trans],[0, 0, 1]])

    x_rot = R @ x
    return x_rot

In [139]:
def project_and_plot(P, X, x, img, path):

    x_proj = cv.transform_and_dehomogenize(P, X)
    x_img = cv.dehomogenize(x)

    theta = -np.pi / 2
    x_trans = 0
    y_trans = np.size(img,0)

    x_proj = rotate_and_translate_P2_points(x_proj, theta, x_trans, y_trans)
    x_img = rotate_and_translate_P2_points(x_img, theta, x_trans, y_trans)

    visible = np.isfinite(x_img)
    x_proj_visible = x_proj * visible

    plot_image_points_projected_points_and_image(x_proj_visible, x_img, img, path)

In [140]:
def transform_project_and_plot(P, X, x, T, view, imfiles, path, name):

    x_view = x[0,view]
    X_transf = cv.transform_and_dehomogenize(T, X)

    P_view = P[view]
    P_transf = cv.transform(P_view, np.linalg.inv(T))

    img1 = r'A2data\data/' + imfiles[0,view][0].strip('\'')
    img = cv.load_image(path+img1)

    project_and_plot(P_transf, X_transf, x_view, img, path+name)

In [141]:
def get_T():
    T1 = np.array([[1,0,0,0],
                   [0,3,0,0],
                   [0,0,1,0],
                   [1/8,1/8,0,1]])
    
    T2 = np.array([[1,0,0,0],
                   [0,1,0,0],
                   [0,0,1,0],
                   [1/16,1/16,0,1]])
    
    return T1, T2

In [142]:
def compute_camera_and_normalized_principal_axis(P, multi=False):

    if multi:
        C_arr = np.array([cv.homogenize(cv.compute_camera_center(P[i])) for i in range(np.size(P))])
        axis_arr = np.array([cv.homogenize(cv.compute_normalized_principal_axis(P[i])) for i in range(np.size(P))])
    else:
        C_arr = np.array([cv.homogenize(cv.compute_camera_center(P))])
        axis_arr = np.array([cv.homogenize(cv.compute_normalized_principal_axis(P))])

    if np.size(C_arr, 0) != 4: # (n,4) => (4,n)
        C_arr = C_arr.T
        axis_arr = axis_arr.T
    
    return C_arr, axis_arr

In [143]:
def transform_cameras(C_arr, axis_arr, T):
    C_transf_arr = cv.transform_and_dehomogenize(T, C_arr)
    axis_transf_arr = cv.transform_and_dehomogenize(T, axis_arr)
    return C_transf_arr, axis_transf_arr

In [147]:
path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-2/'
data = r'A2data\data\compEx1data.mat'

P = cv.convert_mat_to_np(path+data, 'P')[0]
X = cv.dehomogenize(cv.convert_mat_to_np(path+data, 'X'))
x = cv.convert_mat_to_np(path+data, 'x')
imfiles = cv.convert_mat_to_np(path+data, 'imfiles')

C_arr, axis_arr = compute_camera_and_normalized_principal_axis(P, multi=True)

s = 30

name = r'report-images\E1_3D.png'
# plot_cameras_and_3D_points(X, C_arr, axis_arr, s, path+name)

view = 0
P1 = P[view]
x1 = x[0,view]

img1 = r'A2data\data/' + imfiles[0,view][0].strip('\'')
img = cv.load_image(path+img1)

name = r'report-images\E1_2D.png'
project_and_plot(P1, X, x1, img, path+name)

T1, T2 = get_T()
X1 = cv.transform_and_dehomogenize(T1, X)
X2 = cv.transform_and_dehomogenize(T2, X)

C1_arr, axis1_arr = transform_cameras(C_arr, axis_arr, T1)
C2_arr, axis2_arr = transform_cameras(C_arr, axis_arr, T2)

name = r'report-images\E1_3D_T1.png'
# plot_cameras_and_3D_points(X1, C1_arr, axis1_arr, s, path+name)

name = r'report-images\E1_3D_T2.png'
# plot_cameras_and_3D_points(X2, C2_arr, axis2_arr, s, path+name)

view = -1

name = r'report-images\E1_2D_T1.png'
# transform_project_and_plot(P, X, x, T1, view, imfiles, path, name)

name = r'report-images\E1_2D_T2.png'
transform_project_and_plot(P, X, x, T2, view, imfiles, path, name)